# Generic programming

We have seen **duck typing** as a convient abstraction tool for data types:

* As a **user**, we don't have to care about how a specific type, say an `Array`, is implemented. By being an array (i.e. a subtype of `AbstractArray`) it behaves like we expect and we can just use it.
* As a **developer**, as long as we make our objects behave like, say, and `AbstractArray`, we can implement it in whatever way we deem appropriate and it will work with all kinds of algorithms.

Building upon this principle, we also want to formulate our **algorithms** in an abstract way such that it works will all kinds of data types irrespective of their precise implementation (or even meaning). This is generally known as **generic programming**.

From [Wikipedia](https://en.wikipedia.org/wiki/Generic_programming):

> **Generic programming** is a style of computer programming in which algorithms are written in terms of types *to-be-specified-later* that are then *instantiated* when needed for specific types provided as parameters.

## Example 1: Summation

In [45]:
function sum_naive(x)
    s = 0.0
    for xi in x
        s += xi
    end
    return s
end

sum_naive (generic function with 2 methods)

In [46]:
function sum_generic(x)
    s = zero(eltype(x))
    for xi in x
        s += xi
    end
    return s
end

sum_generic (generic function with 2 methods)

In [47]:
using BenchmarkTools

In [48]:
x = rand(100_000);
@btime sum_naive($x);
@btime sum_generic($x);

  93.500 μs (0 allocations: 0 bytes)
  93.500 μs (0 allocations: 0 bytes)


In [49]:
x = rand(Int, 100_000);
@btime sum_naive($x);
@btime sum_generic($x);

  93.541 μs (0 allocations: 0 bytes)
  15.541 μs (0 allocations: 0 bytes)


### Example 2: Vandermonde matrix
(modified from [Steven's Julia intro](https://web.mit.edu/18.06/www/Fall17/1806/julia/Julia-intro.pdf))

\begin{align}V=\begin{bmatrix}1&\alpha _{1}&\alpha _{1}^{2}&\dots &\alpha _{1}^{n-1}\\1&\alpha _{2}&\alpha _{2}^{2}&\dots &\alpha _{2}^{n-1}\\1&\alpha _{3}&\alpha _{3}^{2}&\dots &\alpha _{3}^{n-1}\\\vdots &\vdots &\vdots &\ddots &\vdots \\1&\alpha _{m}&\alpha _{m}^{2}&\dots &\alpha _{m}^{n-1}\end{bmatrix}\end{align}

In [20]:
using PythonCall

In [21]:
np = pyimport("numpy")

Python module: <module 'numpy' from '/Users/carstenbauer/repos/JuliaHLRS22/.CondaPkg/env/lib/python3.10/site-packages/numpy/__init__.py'>

In [22]:
np.vander(1:5, increasing=true)

Python ndarray:
array([[  1,   1,   1,   1,   1],
       [  1,   2,   4,   8,  16],
       [  1,   3,   9,  27,  81],
       [  1,   4,  16,  64, 256],
       [  1,   5,  25, 125, 625]])

The source code for this function is [here](https://github.com/numpy/numpy/blob/v1.16.1/numpy/lib/twodim_base.py#L475-L563). It calls `np.multiply.accumulate` which is implemented in C [here](https://github.com/numpy/numpy/blob/deea4983aedfa96905bbaee64e3d1de84144303f/numpy/core/src/umath/ufunc_object.c#L3678). However, this code doesn't actually perform the computation, it basically only checks types and stuff. The actual kernel that gets called is [here](https://github.com/numpy/numpy/blob/deea4983aedfa96905bbaee64e3d1de84144303f/numpy/core/src/umath/loops.c.src#L1742). This isn't even C code but a template for C code which is used to generate type specific kernels.

Overall, this setup only supports a limited set of types, like `Float64`, `Float32`, and so forth.

Here is our simple generic Julia implementation


In [4]:
function vander(x::AbstractVector{T}) where T
    m = length(x)
    V = Matrix{T}(undef, m, m)
    for j = 1:m
        V[j,1] = one(x[j])
    end
    for i= 2:m
        for j = 1:m
            V[j,i] = x[j] * V[j,i-1]
            end
        end
    return V
end

vander (generic function with 1 method)

In [24]:
vander(1:5)

5×5 Matrix{Int64}:
 1  1   1    1    1
 1  2   4    8   16
 1  3   9   27   81
 1  4  16   64  256
 1  5  25  125  625

#### A quick speed comparison

<details>
  <summary>Show Code</summary>
<br>
    
```julia
using BenchmarkTools, Plots
ns = exp10.(range(1, 4, length=30));

tnp = Float64[]
tjl = Float64[]
for n in ns
    x = 1:n |> collect
    push!(tnp, @belapsed np.vander(\$x) samples=3 evals=1)
    push!(tjl, @belapsed vander(\$x) samples=3 evals=1)
end
plot(ns, tnp./tjl, m=:circle, xscale=:log10, xlab="matrix size", ylab="NumPy time / Julia time", legend=:false)
```
</details>

 <img src="../imgs/vandermonde.svg" alt="drawing" width="600"/>

Note that the clean and concise Julia implementation is **beating numpy's C implementation for small matrices** and is **on-par for large matrix sizes**.

At the same time, the Julia code is *generic* and works for arbitrary types!


In [25]:
vander(Int32[4, 8, 16, 32])

4×4 Matrix{Int32}:
 1   4    16     64
 1   8    64    512
 1  16   256   4096
 1  32  1024  32768

It even works for non-numerical types. The only requirement is that the type has a *one* (identity element) and a multiplication operation defined.


### New "features" emerging from generic programming

#### Symbolic computations

In [2]:
using Symbolics

In [3]:
@variables a b c d e

5-element Vector{Num}:
 a
 b
 c
 d
  e

In [5]:
v = vander([a,b,c,d,e])

5×5 Matrix{Num}:
 1  a   a^2     a^3     a^4
 1  b   b^2     b^3     b^4
 1  c   c^2     c^3     c^4
 1  d   d^2     d^3     d^4
 1   e     e^2     e^3     e^4

In [10]:
substitute(v, Dict(b => 2, d => 4))

5×5 Matrix{Num}:
 1  a   a^2     a^3     a^4
 1  2     4       8      16
 1  c   c^2     c^3     c^4
 1  4    16      64     256
 1   e     e^2     e^3     e^4

#### Arbitrary precision computing

In [27]:
x = rand(BigFloat, 10)

10-element Vector{BigFloat}:
 0.4307640270974499815207965978535850608653035933720816943615073789157893823968412
 0.5326682260798562752348090054011957467209771712345945654899185407917047140162041
 0.1377485169227343699703917518200821630574071782556780670714271956667791896371105
 0.4051566721891543201628641192532849052845650633301904248007311465311606153648328
 0.4925104429337762559135377105405543582997483083558575461696840592740746877901134
 0.4933417271285014483249600817684355915889749932278128139377544081002335134438573
 0.1005106220324690233925243478156836398489221244447668104206712582310932239203369
 0.6252223627360774110538282947388224590177818420127545075491329482269315589319916
 0.07472979855090539582105947875421104267514260639785159930291281993775397943814822
 0.3349145455837027795486395821039475290469982247926209939896419325993560067771694

In [28]:
sum(x)

3.627566941254627260943410970049802496405821105424209023093381688274876871716623

#### Differential equation solving with uncertainty

**Code:**
```julia
using OrdinaryDiffEq, Measurements, Plots

#Half-life of Carbon-14 is 5730 years.
c = 5.730 ± 2

#Setup
u0 = 1.0 ± 0.1
tspan = (0.0, 1.0)

#Define the problem
radioactivedecay(u,p,t) = -c*u

#Pass to solver
prob = ODEProblem(radioactivedecay,u0,tspan)
sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8);

plot(sol.t, sol.u, ylabel="u(t)", xlabel="t", lw=2, legend=false)
```

**Output:**
<img src="../imgs/ode_uncertainty.svg">

**Historical note**: In some sense, **Julia implemented that feature by itself**. The authors of Measurements.jl and DifferentialEquations.jl never had any collabration on this.

# Core messages of this Notebook

* Julia **can be fast.**
* **A function is compiled when called for the first time** with a given set of argument types.
* The are **multiple compilation steps** all of which can be inspected through macros like `@code_warntype`.
* **Code specialization** based on the types of all of the input arguments is important for speed.
* Calculations can be moved to compile-time to make run-time faster.
* In virtually all cases, **explicit type annotations are irrelevant for performance**.
* Type annotations in function signatures define a **type filter/user interface**.
